라이브러리 gensim을 설치하겠습니다

In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 58.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 34.0 MB/s eta 0:00:00


# Word2Vec

In [3]:
import gensim
gensim.__version__

'4.3.2'

In [4]:
!pip install nltk==3.8.1

## 영어 데이터 다운로드 및 전처리

In [5]:
import re
from lxml import etree
import urllib.request
import zipfile
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Word2Vec을 학습하기 위해서 데이터를 다운로드합니다.

TED talk를 구분하는 데이터셋입니다

In [7]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f438a78c2b0>)

In [9]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

현재 영어 텍스트가 content_text에 저장되어져 있습니다. 이에 대해서 NLTK의 sent_tokenize를 통해서 문장을 구분해봅시다.

In [10]:
len(content_text)

24062319

In [11]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [13]:
from tqdm import tqdm

In [15]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in tqdm(sent_text):
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

100%|██████████| 273424/273424 [00:01<00:00, 160722.46it/s]


In [16]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in tqdm(normalized_text)]

100%|██████████| 273424/273424 [00:27<00:00, 10098.08it/s]


총 문장의 개수는 273,424개입니다.

In [17]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [18]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


result => 이중 리스트

## 영어 Word2Vec 훈련시키기

In [19]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.  

vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
window = 컨텍스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
workers = 학습을 위한 프로세스 수  
sg = 0은 CBOW, 1은 Skip-gram.  

In [20]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8416104316711426), ('guy', 0.8220253586769104), ('boy', 0.7731236219406128), ('lady', 0.7690756320953369), ('girl', 0.7375949025154114), ('soldier', 0.7217484712600708), ('gentleman', 0.7211280465126038), ('kid', 0.697986900806427), ('poet', 0.6560426354408264), ('physicist', 0.6410159468650818)]


In [21]:
model.wv["man"]

array([ 1.1690477 , -2.3000562 , -0.12750985,  0.8506692 ,  1.321779  ,
       -0.16566426,  0.47735184,  0.6338332 , -0.63897467,  0.36994684,
       -0.5610864 , -0.5982627 ,  0.21070126, -0.17252913, -0.44207135,
       -0.38002506,  0.3763458 ,  0.4817234 ,  1.1950139 , -1.3788924 ,
       -0.17806627,  1.3669987 ,  0.6135797 , -0.87568235,  1.4338871 ,
        0.31862527, -2.8628678 , -1.0590672 ,  0.48081678, -0.46098468,
       -0.44726095, -0.57253426,  0.79906464, -0.06176937, -0.28506753,
       -0.8231528 , -1.4289908 , -0.25961506, -1.4218746 , -0.18879646,
        0.5119494 , -1.6723247 , -1.1334915 ,  0.81145394, -0.00306537,
       -0.6122859 , -1.4387882 , -1.7483294 , -1.4700736 , -0.13123232,
       -0.3544975 , -1.5006703 , -0.08489141,  0.93261135,  0.08584834,
       -0.8866178 , -0.54769015, -0.504674  , -1.7033166 , -0.3974791 ,
        0.167177  , -0.14560491,  1.1091609 ,  0.6708006 , -2.2936544 ,
        0.54983246, -0.9749112 ,  1.372121  , -0.2282077 ,  1.82

학습한 모델은 저장하고 나중에 불러서 사용할 수 있습니다

In [22]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [23]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8416104316711426), ('guy', 0.8220253586769104), ('boy', 0.7731236219406128), ('lady', 0.7690756320953369), ('girl', 0.7375949025154114), ('soldier', 0.7217484712600708), ('gentleman', 0.7211280465126038), ('kid', 0.697986900806427), ('poet', 0.6560426354408264), ('physicist', 0.6410159468650818)]


In [24]:
# 현재 경로
%pwd

'/workspace/tutorials'

## 한국어 데이터 다운로드 및 전처리

KoNLPy의 OKT 등은 형태소 분석 속도가 너무 느립니다. 그래서 Mecab을 설치하겠습니다.  
단, Mecab은 형태소 분석 속도는 빠르지만 설치하는데 시간이 좀 걸립니다.

mecab는 Windows에서 사용하기 위해서는 Docker로 사용해야 합니다.

https://github.com/koorukuroo/mecab-ko-web 등을 활용해 보세요

In [25]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.8/577.8 kB 13.8 MB/s eta 0:00:0000:01
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1232 sha256=ad9bc0a1cf00a898ec93a0e629fce3f36aa8080fe283b824356b91c83bca71b8
  Stored in directory: /root/.cache/pip/wheels/e6/7f/28/288e44286706c86b174c304c36815d86e959a4eb1f382f7130
Successfully built mecab-python
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [26]:
import urllib.request
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [27]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f4367291c70>)

In [28]:
train_data = pd.read_table('ratings.txt')

In [29]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [30]:
print(len(train_data)) # 리뷰 개수 출력

200000


In [31]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

True


In [32]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [33]:
print(len(train_data)) # 리뷰 개수 출력

199992


In [34]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/tmp/ipykernel_570494/2342004949.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [35]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [36]:
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [37]:
# 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
mecab = Mecab()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [38]:
print(tokenized_data[:3])

[['어릴', '때', '보', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], ['디자인', '배우', '학생', '으로', '외국', '디자이너', '그', '일군', '전통', '통해', '발전', '해', '문화', '산업', '부러웠', '는데', '사실', '우리', '나라', '에서', '그', '어려운', '시절', '끝', '까지', '열정', '지킨', '노라노', '같', '전통', '있', '어', '저', '같', '사람', '꿈', '꾸', '이뤄나갈', '수', '있', '다는', '것', '감사', '합니다'], ['폴리스', '스토리', '시리즈', '부터', '뉴', '까지', '버릴', '께', '하나', '없', '음', '최고']]


## 한국어 Word2Vec 훈련시키기

nested list를 데이터로 넣어주셔야 합니다.

[['나는', '사과를', 먹는다'], ['이', '영화', '는', '재밌어']]

In [39]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [40]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(18134, 100)

In [41]:
print(model.wv.most_similar("최민식"))

[('한석규', 0.8603258728981018), ('안성기', 0.8475167751312256), ('설경구', 0.8402875661849976), ('송강호', 0.8355334401130676), ('전도연', 0.8327798247337341), ('윤제문', 0.8316043019294739), ('김명민', 0.8233888149261475), ('송윤아', 0.8228549361228943), ('성동일', 0.8195948004722595), ('강동원', 0.8143473267555237)]


In [42]:
model.wv['최민식']

array([-0.07690933,  0.10760616, -0.12111061, -0.13174157, -0.31332618,
       -0.24514377,  0.2124354 ,  0.31267712, -0.06190833, -0.02784704,
       -0.10741881, -0.29288897, -0.29051748,  0.0628472 , -0.0861575 ,
        0.08652798, -0.19227837, -0.07352545,  0.39727005, -0.26496777,
        0.3597672 , -0.13533644,  0.19930178,  0.35163465,  0.04949276,
        0.0793315 ,  0.06464131, -0.2765175 ,  0.0992411 , -0.01137746,
        0.32359326,  0.03243385,  0.37222567,  0.12127119, -0.01362356,
        0.26229843,  0.19138081,  0.18363705, -0.32118186, -0.45124513,
       -0.03095764, -0.4557372 ,  0.11061997,  0.3333827 ,  0.25631362,
       -0.2599053 , -0.11440278, -0.3592126 ,  0.283927  , -0.16332577,
       -0.08104603,  0.2560363 , -0.07084211,  0.15483686, -0.19619627,
       -0.09164094, -0.0730757 ,  0.10749123,  0.18058175, -0.12219261,
        0.18514092,  0.35822898, -0.45358577,  0.06325798, -0.27477926,
        0.1682189 ,  0.02918436,  0.2781835 , -0.25433928,  0.43

In [43]:
print(model.wv.most_similar("히어로"))

[('슬래셔', 0.875518798828125), ('호러', 0.8372799754142761), ('무비', 0.8070620894432068), ('하이틴', 0.80275958776474), ('무협', 0.7945577502250671), ('패러디', 0.7936792373657227), ('느와르', 0.7931198477745056), ('최고봉', 0.7926020622253418), ('고어', 0.7917219996452332), ('정통', 0.788837194442749)]


In [46]:
# 영어 모델이 저장된 경로로 이동
# %cd /content

In [45]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('kor_w2v') # 모델 저장

이제 모델은 보관해 두었다가 필요할 때 불러서(load) 사용하실 수 있습니다.

# 과제 : skip-gram을 구현해 보기
https://wikidocs.net/69141 의 `3. 20뉴스그룹 데이터 전처리하기` 를 사용하여 실습 2에서 사용한 뉴스 데이터를 skip-gram with negative sample로 임베딩을 구현해 보세요!